# LIDC IDRI 2D SEGMENTATION WITH TERNARY CLASSES

## Import Libraries

In [1]:
import pandas as pd
import argparse
import os
from collections import OrderedDict
from glob import glob
import yaml
import numpy as np

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms.functional as TF
from torchvision import transforms
import torchsummary as summary

import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, jaccard_score
from tqdm import tqdm

from Unet_new.unet_model import UNet
from UnetNested.Nested_Unet import NestedUNet

## Define Parameters

In [2]:
name = "UNet"           #default = "UNet"; can be NestedUNet
epochs = 100            #default = 400
batch_size = 12          #default = 12
early_stopping = 50     #default = 50
num_workers = 8        #default = 8
optimizer = 'Adam'      #default = 'Adam'; can be SGD
lr = 1e-5               #default = 1e-5
momentum = 0.9          #default = 0.9
weight_decay = 1e-4     #default = 1e-4
nesterov = False        #default = False
augmentation = True     #default = False

## Define Functions

### Dataset

In [3]:
class LidcDataset(Dataset):
    def __init__(self, IMAGES_PATHS, MASK_PATHS, transforms):
        self.image_paths = IMAGES_PATHS
        self.mask_paths = MASK_PATHS
        
        self.transforms = transforms

    def __getitem__(self, index):
        image = np.load(self.image_paths[index])
        mask = np.load(self.mask_paths[index])

        #Make image and mask 3 dimensional
        image = image.reshape(512,512,1)
        mask = mask.reshape(512,512,1)

        #Convert datatype
        mask = mask.astype('uint8')

        #Apply augmentation
        augmented = self.transforms(image=image,mask=mask)
        image = augmented['image']
        mask = augmented['mask']
        mask = mask.reshape([1,512,512])

        image, mask = image.type(torch.FloatTensor), mask.type(torch.FloatTensor)     

        return image, mask
    
    def __len__(self):
        return len(self.image_paths)

In [4]:
transform = A.Compose([
            A.ElasticTransform(alpha=1.1,alpha_affine=0.5,sigma=5,p=0.15),
            A.HorizontalFlip(p=0.5),
            ToTensorV2()
        ])

### Metrics

In [5]:
def sensitivity_metric(target, output):
    tn, fp, fn, tp = confusion_matrix(target, output)
    s0 = tp[0]/(tp[0]+fn[0])
    s1 = tp[1]/(tp[1]+fn[1])
    s2 = tp[2]/(tp[2]+fn[2])
    s3 = tp[3]/(tp[3]+fn[3])
    sensitivity = np.array([s0, s1, s2, s3]) 
    return sensitivity

### Utilities

In [6]:
def str_to_bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class AverageMeter(object):
    #Computes and stores the average and current value
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Get Configuration

In [7]:
if augmentation == True:
    file_name = name + '_with_augmentation'
else:
    file_name = name + '_base'
os.makedirs('model_outputs/{}'.format(file_name), exist_ok=True)
print("Creating directory called ", file_name)

print('-' * 20)
print("Configuration Setting: ")
print("Model: ", name)
print("Max Epochs: ", epochs)
print("Batch Size: ", batch_size)
print("Number of Workers: ", num_workers)
print("Optimizer: ", optimizer)
print("Learning Rate: ", lr)
print("Augmentation: ", augmentation)

Creating directory called  UNet_with_augmentation
--------------------
Configuration Setting: 
Model:  UNet
Max Epochs:  100
Batch Size:  12
Number of Workers:  8
Optimizer:  Adam
Learning Rate:  1e-05
Augmentation:  True


## Create Model

In [8]:
criterion = torch.nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else torch.nn.CrossEntropyLoss()
cudnn.benchmark = True

#Creating the model
print("Creating model...")
if name == 'NestedUNet':
    model = NestedUNet(num_classes=4)
else:
    model = UNet(n_channels=1, n_classes=4)
model = model.cuda() if torch.cuda.is_available() else model

if torch.cuda.device_count() > 1:
    print("We can use ", torch.cuda.device_count(), " GPUs.")
    model = nn.DataParallel(model)

params = filter(lambda p: p.requires_grad, model.parameters())

if optimizer == 'Adam':
    optimizer = optim.Adam(params, lr=lr, weight_decay=weight_decay)
elif optimizer == 'SGD':
    optimizer = optim.SGD(params, lr=lr, momentum=momentum, nesterov=nesterov, weight_decay=weight_decay)
else:
    raise NotImplementedError
    
summary.summary(model,(1,512,512))

Creating model...
Layer (type:depth-idx)                   Output Shape              Param #
├─inconv: 1-1                            [-1, 64, 512, 512]        --
|    └─double_conv: 2-1                  [-1, 64, 512, 512]        --
|    |    └─Sequential: 3-1              [-1, 64, 512, 512]        37,824
├─down: 1-2                              [-1, 128, 256, 256]       --
|    └─Sequential: 2-2                   [-1, 128, 256, 256]       --
|    |    └─MaxPool2d: 3-2               [-1, 64, 256, 256]        --
|    |    └─double_conv: 3-3             [-1, 128, 256, 256]       221,952
├─down: 1-3                              [-1, 256, 128, 128]       --
|    └─Sequential: 2-3                   [-1, 256, 128, 128]       --
|    |    └─MaxPool2d: 3-4               [-1, 128, 128, 128]       --
|    |    └─double_conv: 3-5             [-1, 256, 128, 128]       886,272
├─down: 1-4                              [-1, 512, 64, 64]         --
|    └─Sequential: 2-4                   [-1, 512, 64

Layer (type:depth-idx)                   Output Shape              Param #
├─inconv: 1-1                            [-1, 64, 512, 512]        --
|    └─double_conv: 2-1                  [-1, 64, 512, 512]        --
|    |    └─Sequential: 3-1              [-1, 64, 512, 512]        37,824
├─down: 1-2                              [-1, 128, 256, 256]       --
|    └─Sequential: 2-2                   [-1, 128, 256, 256]       --
|    |    └─MaxPool2d: 3-2               [-1, 64, 256, 256]        --
|    |    └─double_conv: 3-3             [-1, 128, 256, 256]       221,952
├─down: 1-3                              [-1, 256, 128, 128]       --
|    └─Sequential: 2-3                   [-1, 256, 128, 128]       --
|    |    └─MaxPool2d: 3-4               [-1, 128, 128, 128]       --
|    |    └─double_conv: 3-5             [-1, 256, 128, 128]       886,272
├─down: 1-4                              [-1, 512, 64, 64]         --
|    └─Sequential: 2-4                   [-1, 512, 64, 64]         --
|

## Load Dataset

In [9]:
#directory of Images and Masks folders (generated from preprocessing)                                         
IMAGE_DIR = '/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Image/'
MASK_DIR = '/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Mask/'                                                                 

#meta information
meta = pd.read_csv('/scratch1/joseph.portugal/LIDC-IDRI Preprocessed Exp 3/Meta/meta.csv')
meta = meta[meta['patient_diagnosis'] != 0]

#Get train/test label from metadata file
meta['original_image'] = meta['original_image'].apply(lambda x: IMAGE_DIR + "LIDC-IDRI-" + x[:4] + "/" + x + ".npy")
meta['mask_image'] = meta['mask_image'].apply(lambda x: MASK_DIR + "LIDC-IDRI-" + x[:4] + "/" + x + ".npy")


#Split into training and validation
train_meta = meta[meta['data_split']=='Train']
val_meta = meta[meta['data_split']=='Validation']

#Get training images into list
train_image_paths = list(train_meta['original_image'])
train_mask_paths = list(train_meta['mask_image'])

#Get validation images into list
val_image_paths = list(val_meta['original_image'])
val_mask_paths = list(val_meta['mask_image'])

print("*"*50)
print("Original images: {}, masks: {} for training.".format(len(train_image_paths),len(train_mask_paths)))
print("Original images: {}, masks: {} for validation.".format(len(val_image_paths),len(val_mask_paths)))
print("Ratio between Validation and Training is {:2f}".format(len(val_image_paths)/len(train_image_paths)))
print("*"*50)


#Creating custom LIDC dataset
train_dataset = LidcDataset(train_image_paths, train_mask_paths, transforms=transform)
val_dataset = LidcDataset(val_image_paths, val_mask_paths, transforms=transform)

#Creating Dataloader
train_loader = DataLoader(
  train_dataset,
  batch_size=batch_size,
  shuffle=True,
  pin_memory=True,
  drop_last=True,
  num_workers=num_workers
)
val_loader = DataLoader(
  val_dataset,
  batch_size=batch_size,
  shuffle=False,
  pin_memory=True,
  drop_last=False,
  num_workers=num_workers
)

**************************************************
Original images: 980, masks: 980 for training.
Original images: 177, masks: 177 for validation.
Ratio between Validation and Training is 0.180612
**************************************************


## Train the Model

In [10]:
torch.cuda.empty_cache()

In [11]:
log = pd.DataFrame(index=[], columns=['epoch','lr','loss','iou','dice','sensitivity','val_loss','val_iou', 'val_sensitivity'])
# log = pd.DataFrame(index=[], columns=['epoch','lr','loss','iou','dice','val_loss','val_iou'])

best_dice = 0
trigger = 0

for epoch in range(epochs):

    #Model Training
    avg_meters = {'loss': AverageMeter(), 'iou': AverageMeter(), 'dice': AverageMeter(), 'sensitivity': AverageMeter()}
#     avg_meters = {'loss': AverageMeter(), 'iou': AverageMeter(), 'dice': AverageMeter()}
    model.train()
    pbar = tqdm(total=len(train_loader)) #progress bar

    for i, data in enumerate(train_loader):

        input = data[0].cuda()
        target = data[1].cuda()
        output = model(input)

        #Get loss and metric
        target_metric = target.flatten().cpu()
        output_metric = torch.argmax(output, dim=1).flatten().cpu()
        loss = criterion(output, torch.argmax(target, dim=1))
        iou = jaccard_score(target_metric, output_metric, average=None)
        dice = f1_score(target_metric, output_metric, average=None)
        sensitivity = sensitivity_metric(target_metric, output_metric)

        #Calculate the gradient and perform optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Update average metrics
        avg_meters['loss'].update(loss.item(), input.size(0))
        avg_meters['iou'].update(iou, input.size(0))
        avg_meters['dice'].update(dice, input.size(0))
        avg_meters['sensitivity'].update(sensitivity, input.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
            ('dice', avg_meters['dice'].avg),
            ('sensitivity', avg_meters['sensitivity'].avg)
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    train_log = OrderedDict([
        ('loss', avg_meters['loss'].avg),
        ('iou', avg_meters['iou'].avg),
        ('dice', avg_meters['dice'].avg),
        ('sensitivity', avg_meters['sensitivity'].avg)
    ])


    #Model Validation
    val_avg_meters = {'val_loss': AverageMeter(), 'val_iou': AverageMeter(), 'val_dice': AverageMeter(), 'val_sensitivity': AverageMeter()}
#     val_avg_meters = {'val_loss': AverageMeter(), 'val_iou': AverageMeter(), 'val_dice': AverageMeter()}
    model.eval()

    with torch.no_grad():
        val_pbar = tqdm(total=len(val_loader))
        for i, val_data in enumerate(val_loader):

            val_input = val_data[0].cuda()
            val_target = val_data[1].cuda()
            val_output = model(val_input)
            
            val_target_metric = val_target.flatten().cpu()
            val_output_metric = torch.argmax(val_output, dim=1).flatten().cpu()
            val_loss = criterion(val_output, torch.argmax(val_target, dim=1))
            val_iou = jaccard_score(val_target_metric, val_output_metric, average=None)
            val_dice = f1_score(val_target_metric, val_output_metric, average=None)
            val_sensitivity = sensitivity_metric(val_target_metric, val_output_metric)

            val_avg_meters['val_loss'].update(val_loss.item(), val_input.size(0))
            val_avg_meters['val_iou'].update(val_iou, val_input.size(0))
            val_avg_meters['val_dice'].update(val_dice, val_input.size(0))
            val_avg_meters['val_sensitivity'].update(val_sensitivity, val_input.size(0))

            val_postfix = OrderedDict([
                ('val_loss', val_avg_meters['val_loss'].avg),
                ('val_iou', val_avg_meters['val_iou'].avg),
                ('val_dice', val_avg_meters['val_dice'].avg),
                ('val_sensitivity', val_avg_meters['val_sensitivity'].avg)
            ])
            val_pbar.set_postfix(val_postfix)
            val_pbar.update(1)
        val_pbar.close()

    val_log = OrderedDict([
        ('val_loss', val_avg_meters['val_loss'].avg),
        ('val_iou', val_avg_meters['val_iou'].avg),
        ('val_dice', val_avg_meters['val_dice'].avg),
        ('val_sensitivity', val_avg_meters['val_sensitivity'].avg)
    ])
    

    print('Training Epoch {}/{},  Training Loss: {:.4f},  Training DICE: {},  Training IOU: {},  Training Sensitivity: {},  Validation Loss: {:.4f},  Validation DICE: {},  Validation IOU: {},  Validation Sensitivity: {}'.format(
        epoch+1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], train_log['sensitivity'], val_log['val_loss'], val_log['val_dice'], val_log['val_iou'], val_log['val_sensitivity']
    ))
    # print('Training Epoch {}/{},  Training Loss: {:.4f},  Training DICE: {:.4f},  Training IOU: {:.4f},  Validation Loss: {:.4f},  Validation DICE: {:.4f},  Validation IOU: {:.4f}'.format(
    #     epoch+1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], val_log['val_loss'], val_log['val_dice'], val_log['val_iou']
    # ))
#     print('Training Epoch {}/{},  Training Loss: {:.4f},  Training DICE: {},  Training IOU: {},  Validation Loss: {:.4f},  Validation DICE: {},  Validation IOU: {}'.format(
#         epoch+1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], val_log['val_loss'], val_log['val_dice'], val_log['val_iou']
#     ))

    #Save values to csv file
    tmp = pd.Series([
        epoch,
        lr,
        train_log['loss'],
        train_log['iou'],
        train_log['dice'],
        train_log['sensitivity'],
        val_log['val_loss'],
        val_log['val_iou'],
        val_log['val_dice'],
        val_log['val_sensitivity']
    ], index=['epoch', 'lr', 'loss', 'iou', 'dice', 'sensitivity', 'val_loss', 'val_iou', 'val_dice', 'val_sensitivity'])
#     index=['epoch', 'lr', 'loss', 'iou', 'dice', 'val_loss', 'val_iou', 'val_dice'])
    

    log = log.append(tmp, ignore_index=True)
    log.to_csv('model_outputs/{}/log_metrics.csv'.format(file_name), index=False)

    trigger += 1

    #If best DICE score, save the model
    if np.mean(val_log['val_dice']) > best_dice:
        torch.save(model.state_dict(), 'model_outputs/{}/model_metrics.pth'.format(file_name))
        best_dice = np.mean(val_log['val_dice'])
        print("Saved new best model based on DICE metric!")
        trigger = 0
    
    if early_stopping >= 0 and trigger >= early_stopping:
        print("Early stopping.")
        break

    torch.cuda.empty_cache()

 13%|█▎        | 2/15 [00:08<00:51,  3.94s/it, val_loss=1.19, val_iou=[8.71679883e-01 0.00000000e+00 3.10406821e-03 8.26675449e-04], val_dice=[0.93117536 0.         0.00618349 0.00165198], val_sensitivity=[0.92045455 0.09672996 0.24768817 0.40105994]]/tmp/ipykernel_12120/1170807870.py:3: RuntimeWarning: invalid value encountered in long_scalars
  s0 = tp[0]/(tp[0]+fn[0])
 27%|██▋       | 4/15 [00:15<00:41,  3.74s/it, val_loss=1.19, val_iou=[8.75999096e-01 4.71086437e-04 1.55203410e-03 8.75922722e-04], val_dice=[0.93375649 0.0009404  0.00309174 0.00175029], val_sensitivity=[       nan 0.54836498 0.62384409 0.70052997]]/tmp/ipykernel_12120/1170807870.py:3: RuntimeWarning: invalid value encountered in long_scalars
  s0 = tp[0]/(tp[0]+fn[0])
/tmp/ipykernel_12120/1170807870.py:4: RuntimeWarning: invalid value encountered in long_scalars
  s1 = tp[1]/(tp[1]+fn[1])
/tmp/ipykernel_12120/1170807870.py:5: RuntimeWarning: invalid value encountered in long_scalars
  s2 = tp[2]/(tp[2]+fn[2])
/tmp/i

Training Epoch 1/100,  Training Loss: 1.2583,  Training DICE: [9.07770616e-01 5.64087031e-04 6.10417563e-03 2.27337243e-03],  Training IOU: [8.31553101e-01 2.82317894e-04 3.06457467e-03 1.13835248e-03],  Training Sensitivity: [0.2917769  0.35687026 0.38858356 0.41027167],  Validation Loss: 1.1988,  Validation DICE: [9.20825658e-01 2.74479782e-03 5.04404216e-03 4.74655422e-04],  Validation IOU: [8.55119945e-01 1.39049913e-03 2.53345583e-03 2.37538365e-04],  Validation Sensitivity: [nan nan nan nan]
Saved new best model based on DICE metric!


 27%|██▋       | 4/15 [00:15<00:41,  3.79s/it, val_loss=1.15, val_iou=[9.11209066e-01 4.38881832e-04 2.89095585e-03 1.41297258e-03], val_dice=[9.53500276e-01 8.76225427e-04 5.74229797e-03 2.82183019e-03], val_sensitivity=[0.87804878 0.55773095 0.56607636 0.74056745]]/tmp/ipykernel_12120/1170807870.py:3: RuntimeWarning: invalid value encountered in long_scalars
  s0 = tp[0]/(tp[0]+fn[0])
/tmp/ipykernel_12120/1170807870.py:4: RuntimeWarning: invalid value encountered in long_scalars
  s1 = tp[1]/(tp[1]+fn[1])
/tmp/ipykernel_12120/1170807870.py:5: RuntimeWarning: invalid value encountered in long_scalars
  s2 = tp[2]/(tp[2]+fn[2])
/tmp/ipykernel_12120/1170807870.py:6: RuntimeWarning: invalid value encountered in long_scalars
  s3 = tp[3]/(tp[3]+fn[3])
 33%|███▎      | 5/15 [00:19<00:37,  3.74s/it, val_loss=1.15, val_iou=[9.13660933e-01 7.11042531e-04 2.31276468e-03 1.13037807e-03], val_dice=[0.95484255 0.00141956 0.00459384 0.00225746], val_sensitivity=[nan nan nan nan]]                  

Training Epoch 2/100,  Training Loss: 1.1809,  Training DICE: [0.93356233 0.00101442 0.00532933 0.0032123 ],  Training IOU: [8.75525857e-01 5.08337082e-04 2.67418636e-03 1.60968582e-03],  Training Sensitivity: [0.38283649 0.39185259 0.39379956 0.37978425],  Validation Loss: 1.1615,  Validation DICE: [9.33612978e-01 2.17449317e-03 5.86758477e-03 7.65242085e-04],  Validation IOU: [8.77463499e-01 1.09686045e-03 2.95487213e-03 3.83179005e-04],  Validation Sensitivity: [nan nan nan nan]
Saved new best model based on DICE metric!


 27%|██▋       | 4/15 [00:14<00:40,  3.67s/it, val_loss=1.13, val_iou=[9.29577341e-01 5.64624102e-04 3.56500953e-03 2.98256019e-03], val_dice=[0.96347193 0.0011267  0.00707612 0.00594617], val_sensitivity=[0.81887823 0.52803437 0.51447536 0.70889265]]/tmp/ipykernel_12120/1170807870.py:3: RuntimeWarning: invalid value encountered in long_scalars
  s0 = tp[0]/(tp[0]+fn[0])
/tmp/ipykernel_12120/1170807870.py:4: RuntimeWarning: invalid value encountered in long_scalars
  s1 = tp[1]/(tp[1]+fn[1])
/tmp/ipykernel_12120/1170807870.py:5: RuntimeWarning: invalid value encountered in long_scalars
  s2 = tp[2]/(tp[2]+fn[2])
/tmp/ipykernel_12120/1170807870.py:6: RuntimeWarning: invalid value encountered in long_scalars
  s3 = tp[3]/(tp[3]+fn[3])
 33%|███▎      | 5/15 [00:18<00:36,  3.64s/it, val_loss=1.13, val_iou=[9.30442609e-01 8.24042010e-04 2.85200762e-03 2.38604815e-03], val_dice=[0.96394201 0.00164466 0.00566089 0.00475694], val_sensitivity=[nan nan nan nan]]                            /tmp/i

...

Training Epoch 5/100,  Training Loss: 1.1217,  Training DICE: [0.97187308 0.00100075 0.00721736 0.00577285],  Training IOU: [9.45334551e-01 5.01332748e-04 3.63053956e-03 2.90102894e-03],  Training Sensitivity: [0.40131623 0.35078457 0.39121892 0.41512058],  Validation Loss: 1.1102,  Validation DICE: [0.97494019 0.00287737 0.00418529 0.00124983],  Validation IOU: [9.52104384e-01 1.46291197e-03 2.10956904e-03 6.27396638e-04],  Validation Sensitivity: [nan nan nan nan]
Saved new best model based on DICE metric!


 32%|███▏      | 26/81 [03:16<05:44,  6.26s/it, loss=1.11, iou=[9.64158743e-01 2.48300960e-04 2.75598875e-03 4.92791174e-04], dice=[9.81748735e-01 4.95506275e-04 5.48217615e-03 9.83362352e-04], sensitivity=[0.40131175 0.23052601 0.34540821 0.26078568]]/tmp/ipykernel_12120/1170807870.py:6: RuntimeWarning: invalid value encountered in long_scalars
  s3 = tp[3]/(tp[3]+fn[3])
 42%|████▏     | 34/81 [04:23<06:21,  8.11s/it, loss=1.11, iou=[9.65645724e-01 1.95383995e-04 2.74534013e-03 4.11519191e-04], dice=[9.82517749e-01 3.89928082e-04 5.46160243e-03 8.21288629e-04], sensitivity=[0.37719387 0.2067843  0.31714819        nan]]/tmp/ipykernel_12120/1170807870.py:6: RuntimeWarning: invalid value encountered in long_scalars
  s3 = tp[3]/(tp[3]+fn[3])
 49%|████▉     | 40/81 [05:08<05:32,  8.12s/it, loss=1.11, iou=[9.66548946e-01 2.76276322e-04 2.56535322e-03 5.19930926e-04], dice=[9.82984502e-01 5.51352989e-04 5.10426994e-03 1.03659566e-03], sensitivity=[0.37046232 0.24243332 0.3391185         nan

 95%|█████████▌| 77/81 [09:35<00:29,  7.34s/it, loss=1.11, iou=[9.72107403e-01 3.00180085e-04 2.81296684e-03 9.63690663e-04], dice=[9.85843588e-01 5.98893525e-04 5.58022893e-03 1.90818255e-03], sensitivity=[0.38692188        nan        nan        nan]]/tmp/ipykernel_12120/1170807870.py:4: RuntimeWarning: invalid value encountered in long_scalars
  s1 = tp[1]/(tp[1]+fn[1])
/tmp/ipykernel_12120/1170807870.py:5: RuntimeWarning: invalid value encountered in long_scalars
  s2 = tp[2]/(tp[2]+fn[2])
/tmp/ipykernel_12120/1170807870.py:6: RuntimeWarning: invalid value encountered in long_scalars
  s3 = tp[3]/(tp[3]+fn[3])
 96%|█████████▋| 78/81 [09:44<00:23,  7.81s/it, loss=1.11, iou=[9.72243414e-01 2.96331623e-04 2.77690316e-03 9.51335654e-04], dice=[9.85913321e-01 5.91215402e-04 5.50868753e-03 1.88371867e-03], sensitivity=[0.38210946        nan        nan        nan]]/tmp/ipykernel_12120/1170807870.py:4: RuntimeWarning: invalid value encountered in long_scalars
  s1 = tp[1]/(tp[1]+fn[1])
/tmp

 93%|█████████▎| 14/15 [00:51<00:03,  3.63s/it, val_loss=1.11, val_iou=[9.42536579e-01 1.28231094e-04 8.36955657e-04 0.00000000e+00], val_dice=[9.68841649e-01 2.56195776e-04 1.66677609e-03 0.00000000e+00], val_sensitivity=[nan nan nan nan]]

ValueError: not enough values to unpack (expected 4, got 3)